# IsCyclic

In [1]:
import torch
from torch_geometric.loader import DataLoader

from GNN import GCN_IsCyclic
from gen_dataset import create_isCyclic_dataset

## Data

In [2]:
dataset, data_objs = create_isCyclic_dataset(saved=True)
print(dataset[0].x)
print(dataset[0].edge_index)
print(dataset[0].y)

# dataset = dataset.shuffle()

Dataset size:  951
tensor([[1.],
        [1.],
        [1.]])
tensor([[0, 0, 1, 1, 2, 2],
        [1, 2, 0, 2, 0, 1]])
tensor(1., dtype=torch.float64)


In [3]:
train_dataset = dataset[:int(0.8 * len(dataset))]
test_dataset = dataset[int(0.8 * len(dataset)):]

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [4]:
for data in train_loader:
    print(data)
    break

DataBatch(x=[2180, 1], edge_index=[2, 8822], y=[64], batch=[2180], ptr=[65])


## Model

In [5]:
model = GCN_IsCyclic(
    in_features=dataset.num_node_features,
    h_features=64,
    n_classes=2
)

In [6]:
print(model)

GCN_IsCyclic(
  (conv1): GraphConvolution (1 -> 64)
  (conv2): GraphConvolution (64 -> 64)
  (conv3): GraphConvolution (64 -> 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


## Train

In [7]:
for data in train_loader:
    print(data)
    break

DataBatch(x=[1978, 1], edge_index=[2, 10758], y=[64], batch=[1978], ptr=[65])


In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y.long())  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()
    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y.long()).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.

In [9]:
best_test_acc = 0.0
for epoch in range(1, 101):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
    if test_acc >= best_test_acc:
        best_test_acc = test_acc
        best_model_params = model.state_dict()
        print("Checkpoint saved!")

Epoch: 001, Train Acc: 0.8197, Test Acc: 0.3979
Checkpoint saved!
Epoch: 002, Train Acc: 0.6803, Test Acc: 0.3246
Epoch: 003, Train Acc: 0.7750, Test Acc: 0.4188
Checkpoint saved!
Epoch: 004, Train Acc: 0.9132, Test Acc: 0.9895
Checkpoint saved!
Epoch: 005, Train Acc: 0.9224, Test Acc: 0.9895
Checkpoint saved!
Epoch: 006, Train Acc: 0.9211, Test Acc: 1.0000
Checkpoint saved!
Epoch: 007, Train Acc: 0.9908, Test Acc: 0.9948
Epoch: 008, Train Acc: 0.9500, Test Acc: 1.0000
Checkpoint saved!
Epoch: 009, Train Acc: 0.9474, Test Acc: 0.9895
Epoch: 010, Train Acc: 0.9711, Test Acc: 0.9948
Epoch: 011, Train Acc: 0.9434, Test Acc: 0.9895
Epoch: 012, Train Acc: 0.9697, Test Acc: 1.0000
Checkpoint saved!
Epoch: 013, Train Acc: 0.9526, Test Acc: 0.9948
Epoch: 014, Train Acc: 0.9461, Test Acc: 0.9895
Epoch: 015, Train Acc: 0.9224, Test Acc: 1.0000
Checkpoint saved!
Epoch: 016, Train Acc: 0.9513, Test Acc: 0.9948
Epoch: 017, Train Acc: 0.9553, Test Acc: 0.9948
Epoch: 018, Train Acc: 1.0000, Test Acc:

KeyboardInterrupt: 